## How to generate function arguments

In [ ]:
!pip install scipy
!pip install tenacity
!pip install tiktoken
!pip install termcolor 
!pip install openai
!pip install requests

In [2]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo-0613"
openai_key = ""
openai.api_key = openai_key

In [3]:
%load_ext autoreload
%autoreload 2

In [27]:
from repo_gpt.agents.repo_comprehender import RepoUnderstandingAgent
from repo_gpt.agents.base_agent import BaseAgent
from repo_gpt.agents.simple_memory_store import MemoryStore
from repo_gpt.logging_config import configure_logging
import logging

configure_logging(logging.INFO)
root_path = Path("/Users/shrutipatel/projects/work/repo-gpt/")
repo_agent = RepoUnderstandingAgent("Where should I add tests for the new file handler I'm writing? Do I need to create a new test file? If so, where?",
                                    root_path,
                                   openai_key = openai_key,
                                   debug=True)
plan = repo_agent.process_messages()
plan

Searching the codebase for: test file for file handler


'- Create a new file named `test_generic_code_file_handler.py` in the `test/file_handler` directory.\n- Import the necessary modules and classes for testing.\n- Write unit tests to verify the functionality of the `GenericCodeFileHandler` class.\n- Execute the tests and ensure that they pass.'

In [ ]:
from repo_gpt.agents.code_writer import CodeWritingAgent
from repo_gpt.agents.central_intelligence import CentralIntelligenceAgent

root_path = Path("/Users/shrutipatel/projects/work/repo-gpt/")
embedding_file_path =  root_path / ".repo_gpt/code_embeddings.pkl"
writer_agent = CodeWritingAgent(plan, 
                                root_path,
                                    embedding_file_path,
                                   openai_key = openai_key,
                                debug=True)
result = writer_agent.process_messages()
result

In [ ]:
from pathlib import Path
from repo_gpt.agents.central_intelligence import CentralIntelligenceAgent

root_path = Path("/Users/shrutipatel/projects/work/repo-gpt/")
embedding_file_path =  root_path / ".repo_gpt/code_embeddings.pkl"

ci_agent = CentralIntelligenceAgent("Where should I add tests for the new file handler I'm writing? Do I need to create a new test file? If so, where?",
                                    root_path,
                                    embedding_file_path,
                                   openai_key = openai_key,
                                   debug=True)
output = ci_agent.process_messages()

In [ ]:
%debug

In [ ]:
# ## TODO
# - Update initial user query so we include results from semantic search
# - Update system prompt and or the function json to include the output schema of the returned code_embedding csv rows
# - Update sep for CSV output to a more uncommon character since commas are frequently used in code? (This may not be necessary because converting to csv usually ex
# - Add a function where the writeragent can ask the repounderstandingagent
# - Update function so you search for files -- this will decrease the token size because we won't send the all the files every time and this will handle larger codebases with greater ease
# - Use COD to ensure messages are being compressed while retaining all relevant info
# - How can I incorporate reflexion?